In [1]:
import pdfplumber
import os
import re
import pandas as pd
import numpy as np

path = "data/Key_Stats/"
files = os.listdir(path)

def read_pages(dir):
    with pdfplumber.open(dir) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

def full_text(text):
    text = re.sub(r'[^\x00-\x7F]+',' ', text) # only english
    text = re.sub(r'[\n\t]+', ' ', text) # delete '\n' and '\t'
    text = re.sub(r' +', ' ', text) # delete multiple spaces
    return text

def file_id(file):
    pattern = r'[A-Z]{3,3}[0-9]{3,3}'
    return re.findall(pattern, file)

In [15]:
pattern1 = r'day|daily'
pattern2 = r'week|weekly'
pattern3 = r"month|month's"
fund_freq = pd.DataFrame(columns=['file_id', 'freq'])


for i in range(len(files)):
#for i in range(100):
    doc_dir = path + files[i]
    raw = read_pages(doc_dir)
    text = full_text(raw).lower()
    ID = file_id(files[i])
    
    freq_loc = text.find('dealing frequency')
    text_freq = text[freq_loc:freq_loc+100]
    #fee_list = ['subscription', 'switching', 'redemption', 'investment management', 'trustee', 'performance', 'admin']
    #fee_table = pd.DataFrame(columns = ['id'] + fee_list)
    freq_fund = pd.DataFrame(columns=['file_id', 'freq'])
    freq_fund['file_id'] = ID
    if re.search(pattern3, text_freq):
        freq_fund['freq'] = 'monthly'
    elif re.search(pattern2, text_freq):
        freq_fund['freq'] = 'weekly'
    elif re.search(pattern1, text_freq):
        freq_fund['freq'] = 'daily'
    
    
    fund_freq = pd.concat([fund_freq, freq_fund], axis=0)
    

In [13]:
for i in range(len(files)):
    doc_dir = path + files[i]
    if file_id(files[i]) == ['AZU904']:
        print(i)

808


In [14]:
i = 808
doc_dir = path + files[i]
raw = read_pages(doc_dir)
text = full_text(raw).lower()
ID = file_id(files[i])

freq_loc = text.find('dealing frequency')
text_freq = text[freq_loc:freq_loc+100]
#fee_list = ['subscription', 'switching', 'redemption', 'investment management', 'trustee', 'performance', 'admin']
#fee_table = pd.DataFrame(columns = ['id'] + fee_list)
freq_fund = pd.DataFrame(columns=['file_id', 'freq'])
freq_fund['file_id'] = ID
if re.search(pattern3, text_freq):
    print('monthly')
elif re.search(pattern2, text_freq):
    print('weekly')
elif re.search(pattern1, text_freq):
    print('daily')

monthly


In [17]:
fund_freq.to_csv('data/fund_freq.csv', index=False)